## Imports

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain import hub
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain_openai import OpenAIEmbeddings
from pydantic.v1 import BaseModel, Field, EmailStr
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function

## Define Langchain Utilitites

In [3]:
class EmailInput(BaseModel):
    name: str = Field(..., description="The name of the person.")
    email: EmailStr = Field(..., description="The email of the person.")
    message: str = Field(..., description="The message content.")


@tool(args_schema=EmailInput)
def send_email(name: str, email: str, message: str) -> dict:
    """
    Send an email or a message.
    For this function to work, you need to ask the user for their email and name if they haven't provided it in the chat.
    """
    return f"Email sent to {name} at {email} with message: {message}"

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Chatbot

In [4]:
EMBEDDING_FUNCTION = OpenAIEmbeddings(model=os.getenv("EMBEDDING_TYPE"))
EMBEDDING_FUNCTION

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x106f87a60>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x10713e020>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [5]:
PINECONE_VS = PineconeVectorStore(index_name=os.getenv("PINECONE_INDEX_NAME"), embedding=EMBEDDING_FUNCTION)
PINECONE_VS

In [6]:
search_kwargs_vs = {
    "k": int(os.getenv("TOP_K")),
}
if os.getenv("SEARCH_TYPE") == "mmr":
    search_kwargs_vs["fetch_k"] = int(os.getenv("FETCH_K"))
    search_kwargs_vs["lambda_multiplier"] = os.getenv("LAMBDA_MULTIPLIER")
RETRIEVER = PINECONE_VS.as_retriever(search_type=os.getenv("SEARCH_TYPE"), search_kwargs=search_kwargs_vs)
RETRIEVER

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x107178100>, search_kwargs={'k': 4})

In [7]:
PROMPT = hub.pull(os.getenv("LLM_RAG_PROMPT_NAME"))
PROMPT

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'portfolio-rag-prompt', 'lc_hub_commit_hash': 'bc88d0833f8815f523fee4beb9d15e2f3d8bea4816b2b9e693d2ba179d89434e'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='You are an assistant for question-answering tasks and your name is Harpy. Always answer questions as a third person. Use the following pieces of retrieved context to answer the question. If the question is directly directed to you, for example, if the questions are "How are you?", "Who are you?", "Tell me about yourself.", etc.  just say, "Hey! I am Harpy, your chat assistant, Please ask questions about Nikhil. I can answer them for you :-)". If you don\'t know the answer, just say "I am sorry, please contact Nikhil at  nikhilbo@kamath.work. He will be happy to answer your questions.". Where ever required and at relevant places, add emojis as well in the generated sentenc

In [8]:
TOOLS = [send_email]
FUNCTIONS = [convert_to_openai_function(t) for t in TOOLS]
FUNCTIONS

[{'name': 'send_email',
  'description': "Send an email or a message.\nFor this function to work, you need to ask the user for their email and name if they haven't provided it in the chat.",
  'parameters': {'type': 'object',
   'properties': {'name': {'description': 'The name of the person.',
     'type': 'string'},
    'email': {'description': 'The email of the person.',
     'type': 'string',
     'format': 'email'},
    'message': {'description': 'The message content.', 'type': 'string'}},
   'required': ['name', 'email', 'message']}}]

In [9]:
model_kwargs = {
    "top_p": float(os.getenv("LLM_TOP_P")),
    "frequency_penalty": float(os.getenv("LLM_FREQUENCY_PENALTY")),
    "presence_penalty": float(os.getenv("LLM_PRESENCE_PENALTY")),
}
LLM = ChatOpenAI(
    model=os.getenv("LLM_MODEL_NAME"),
    temperature=float(os.getenv("LLM_TEMPERATURE")),
    model_kwargs=model_kwargs
).bind(functions=FUNCTIONS)
LLM

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1073449d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x107346140>, temperature=1.0, model_kwargs={'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0}, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'functions': [{'name': 'send_email', 'description': "Send an email or a message.\nFor this function to work, you need to ask the user for their email and name if they haven't provided it in the chat.", 'parameters': {'type': 'object', 'properties': {'name': {'description': 'The name of the person.', 'type': 'string'}, 'email': {'description': 'The email of the person.', 'type': 'string', 'format': 'email'}, 'message': {'description': 'The message content.', 'type': 'string'}}, 'required': ['name', 'email', 'message']}}]})

In [18]:
CHAIN = (
    temp
    | RunnablePassthrough.assign(context=(lambda x: x["question"]) | RETRIEVER | (lambda docs: "\n\n".join(doc.page_content for doc in docs)))
    | RunnablePassthrough.assign(question=(lambda x: x["question"]))
    | PROMPT
    | LLM
    | OpenAIFunctionsAgentOutputParser()
)
CHAIN

RunnableLambda(temp)
| RunnableAssign(mapper={
    context: RunnableLambda(...)
             | VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x107178100>, search_kwargs={'k': 4})
             | RunnableLambda(...)
  })
| RunnableAssign(mapper={
    question: RunnableLambda(...)
  })
| ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'portfolio-rag-prompt', 'lc_hub_commit_hash': 'bc88d0833f8815f523fee4beb9d15e2f3d8bea4816b2b9e693d2ba179d89434e'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='You are an assistant for question-answering tasks and your name is Harpy. Always answer questions as a third person. Use the following pieces of retrieved context to answer the question. If the question is directly directed to you, for example, if the questions are "How are you?", "Who are you

In [19]:
result = CHAIN.invoke({"question": "Tell me about nikhil's work experience"})
result

{'question': "Tell me about nikhil's work experience"}


AgentFinish(return_values={'output': 'Nikhil has nearly six years of total experience in coding vastly in AI. He has worked as a Machine Learning Engineer at Insureka for almost two years and was also the CTO of a startup called BhavamAI. Additionally, he has worked as a research assistant throughout his Masters program for two years and interned at DragonfruitAI for an entire semester. Nikhil has experience in roles such as CTO, Software Development Engineer (SDE), ML Engineer, and research assistant.'}, log='Nikhil has nearly six years of total experience in coding vastly in AI. He has worked as a Machine Learning Engineer at Insureka for almost two years and was also the CTO of a startup called BhavamAI. Additionally, he has worked as a research assistant throughout his Masters program for two years and interned at DragonfruitAI for an entire semester. Nikhil has experience in roles such as CTO, Software Development Engineer (SDE), ML Engineer, and research assistant.')